In [318]:
import pandas as pd
import torch
import PIL.Image
from torchvision import transforms
import  numpy as np
import  torch
import torchvision
import torch.nn.functional as F
from torch import  nn
from torch import optim

In [319]:

data_file = "data/lungdatasets.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)

In [320]:
# csv使用分号分割，seq指定分隔符为；
data = pd.read_csv(data_file,sep=';')

# 住院时长周数为值
outputs = data.iloc[:, 25]

# 定义特征
all_features = data.iloc[:, 1:25]
strlist = ['性别(1=男，2=女）','入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床转归（0=临床治愈，1=病毒转阴）','首诊时有无发热（0=否，1=是）','以前曾是否患过肺炎（0=否，1=是）','病灶位置是否右上（1=是，0=否）','病灶位置是否右中（1=是，0=否）','病灶位置是否右下（1=是，0=否）','病灶位置是否左上（1=是，0=否）','病灶位置是否左下（1=是，0=否）']
all_features[strlist] = all_features[strlist].astype(np.str)
print(all_features.dtypes)


年龄(岁)                                      float64
性别(1=男，2=女）                                 object
入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）     object
出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）     object
出院时临床转归（0=临床治愈，1=病毒转阴）                      object
首次查体体温（℃）                                  float64
首诊时有无发热（0=否，1=是）                            object
身高（cm）                                     float64
体重(kg)                                     float64
既往史评分                                        int64
以前曾是否患过肺炎（0=否，1=是）                          object
ICU时长（天）                                     int64
呼吸机时长（小时）                                  float64
病灶位置是否右上（1=是，0=否）                           object
病灶位置是否右中（1=是，0=否）                           object
病灶位置是否右下（1=是，0=否）                           object
病灶位置是否左上（1=是，0=否）                           object
病灶位置是否左下（1=是，0=否）                           object
首次检查病灶体积总和（cm?）                            float64
首次检查全部病灶占全肺体积占比(%)             

In [321]:
# for i in all_features['身高（cm）']:
#     if(i==0):
#         i = None
#         print(i)

#修改数值特征中的所有0值为NAN
zero_2_nan_Str = ['身高（cm）','体重(kg)','首次检查病灶体积总和（cm?）','首次检查全部病灶占全肺体积占比(%)','首次检查全部病灶的平均密度（HU）','末次检查病灶体积总和（cm?）','末次检查全部病灶占全肺体积占比(%)','末次检查全部病灶的平均密度（HU）']
all_features[zero_2_nan_Str] = all_features[zero_2_nan_Str].replace(0,np.nan)
all_features

,年龄(岁),性别(1=男，2=女）,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）,出院时临床转归（0=临床治愈，1=病毒转阴）,首次查体体温（℃）,首诊时有无发热（0=否，1=是）,身高（cm）,体重(kg),既往史评分,以前曾是否患过肺炎（0=否，1=是）,ICU时长（天）,呼吸机时长（小时）,病灶位置是否右上（1=是，0=否）,病灶位置是否右中（1=是，0=否）,病灶位置是否右下（1=是，0=否）,病灶位置是否左上（1=是，0=否）,病灶位置是否左下（1=是，0=否）,首次检查病灶体积总和（cm?）,首次检查全部病灶占全肺体积占比(%),首次检查全部病灶的平均密度（HU）,末次检查病灶体积总和（cm?）,末次检查全部病灶占全肺体积占比(%),末次检查全部病灶的平均密度（HU）
0,25.0,1,1.0,2.0,1.0,38.2,1.0,176.0,72.0,2,0.0,0,0.00,0,0,1,1,1.0,172.04,3.68,-335.00,NaN,NaN,NaN
1,29.0,1,2.0,2.0,1.0,36.5,0.0,165.0,70.0,0,0.0,0,0.00,0,0,1,0,0.0,26.49,0.53,-621.00,NaN,NaN,NaN
2,30.0,1,2.0,2.0,1.0,36.0,0.0,175.0,67.5,0,0.0,0,0.00,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,41.0,2,0.0,0.0,1.0,36.5,1.0,156.0,72.5,0,0.0,0,0.00,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,29.0,2,1.0,2.0,1.0,39.0,1.0,160.0,70.0,0,0.0,0,0.00,0,0,1,1,1.0,44.36,1.20,-557.00,NaN,NaN,NaN
5,22.0,1,1.0,1.0,1.0,36.0,1.0,162.0,47.0,1,0.0,0,0.00,0,0,0,0,0.0,NaN,NaN,NaN,4.82,0.09,-590.00
6,61.0,2,3.0,3.0,1.0,37.8,1.0,165.0,66.5,1,0.0,9,201.38,1,1,1,1,1.0,479.46,25.19,-346.00,NaN,NaN,NaN
7,62.0,1,2.0,2.0,1.0,37.6,1.0,176.0,62.0,1,0.0,11,519.74,1,1,1,1,1.0,808.39,22.30,-458.00,2.60,0.07,-700.00
8,62.0,2,2.0,2.0,1.0,37.8,1.0,160.0,73.0,1,0.0,0,0.00,1,1,1,1,1.0,244.55,5.61,-656.00,6.97,0.16,-766.00
9,48.0,2,1.0,1.0,1.0,37.5,1.0,NaN,NaN,0,0.0,0,0.00,0,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [322]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

# print(numeric_features[1])
# print(all_features[numeric_features])

all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features

,年龄(岁),首次查体体温（℃）,身高（cm）,体重(kg),既往史评分,ICU时长（天）,呼吸机时长（小时）,首次检查病灶体积总和（cm?）,首次检查全部病灶占全肺体积占比(%),首次检查全部病灶的平均密度（HU）,末次检查病灶体积总和（cm?）,末次检查全部病灶占全肺体积占比(%),末次检查全部病灶的平均密度（HU）,性别(1=男，2=女）_1,性别(1=男，2=女）_2,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_0.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_1.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_2.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_3.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_4.0,入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_nan,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_0.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_1.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_2.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_3.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_4.0,出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）_nan,出院时临床转归（0=临床治愈，1=病毒转阴）_0.0,出院时临床转归（0=临床治愈，1=病毒转阴）_1.0,出院时临床转归（0=临床治愈，1=病毒转阴）_nan,首诊时有无发热（0=否，1=是）_0.0,首诊时有无发热（0=否，1=是）_1.0,首诊时有无发热（0=否，1=是）_nan,以前曾是否患过肺炎（0=否，1=是）_0.0,以前曾是否患过肺炎（0=否，1=是）_1.0,以前曾是否患过肺炎（0=否，1=是）_nan,病灶位置是否右上（1=是，0=否）_0,病灶位置是否右上（1=是，0=否）_1,病灶位置是否右中（1=是，0=否）_0,病灶位置是否右中（1=是，0=否）_1,病灶位置是否右下（1=是，0=否）_0,病灶位置是否右下（1=是，0=否）_1,病灶位置是否左上（1=是，0=否）_0,病灶位置是否左上（1=是，0=否）_1,病灶位置是否左下（1=是，0=否）_0.0,病灶位置是否左下（1=是，0=否）_1.0,病灶位置是否左下（1=是，0=否）_nan
0,-0.942885,1.080185,0.805554,0.234401,2.316212,-0.196760,-0.145085,-0.093414,-0.023783,1.443427,0.000000,0.000000,0.000000,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0
1,-0.690293,-0.915005,-0.066029,0.110304,-0.423394,-0.196760,-0.145085,-0.528198,-0.110194,-0.801605,0.000000,0.000000,0.000000,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0
2,-0.627145,-1.501826,0.726319,-0.044819,-0.423394,-0.196760,-0.145085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
3,0.067482,-0.915005,-0.779142,0.265426,-0.423394,-0.196760,-0.145085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
4,-0.690293,2.019098,-0.462203,0.110304,-0.423394,-0.196760,-0.145085,-0.474817,-0.091815,-0.299220,0.000000,0.000000,0.000000,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0
5,-1.132328,-1.501826,-0.303733,-1.316822,0.946409,-0.196760,-0.145085,0.000000,0.000000,0.000000,-0.423384,-0.299832,0.355788,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0
6,1.330441,0.610728,-0.066029,-0.106868,0.946409,1.926322,1.818024,0.824906,0.566283,1.357079,0.000000,0.000000,0.000000,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
7,1.393589,0.376000,0.805554,-0.386088,0.946409,2.398118,4.921486,1.807480,0.487004,0.477906,-0.433331,-0.301424,-0.523772,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
8,1.393589,0.610728,-0.462203,0.296450,0.946409,-0.196760,-0.145085,0.123187,0.029161,-1.076347,-0.413752,-0.294259,-1.051509,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0
9,0.509518,0.258636,0.000000,0.000000,-0.423394,-0.196760,-0.145085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0


In [323]:
#划分测试集 训练集
splitor = int(len(all_features)*0.8)
train_data = all_features[:splitor]
test_data = all_features[splitor:]

train_inputs  = torch.tensor(train_data.values)
train_outputs = torch.tensor(outputs[:splitor].values)

test_inputs  = torch.tensor(test_data.values)
test_outputs = torch.tensor(outputs[splitor:].values)


# #输入
# data = torch.tensor(all_features.values)
# target = torch.tensor(outputs.values)
#
# data_dim = data.shape[1]
# # target_dim = target.shape[1]
# print(data)
# print(data.shape)
# print(data_dim,target.shape)
# target.unique()

In [324]:
from torch.utils.data import Dataset


class LungDataset(Dataset):

    def __init__(self,inputs,outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __getitem__(self, idx):
        vector = self.inputs[idx]
        label = self.outputs[idx]
        return vector,label

    def __len__(self):
        return len(self.outputs)

lungdataset = LungDataset(train_inputs,train_outputs)
testdataset = LungDataset(test_inputs,test_outputs)
x,y = lungdataset[0]
x,y



(tensor([-0.9429,  1.0802,  0.8056,  0.2344,  2.3162, -0.1968, -0.1451, -0.0934,
         -0.0238,  1.4434,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000],
        dtype=torch.float64),
 tensor(4))

In [325]:
#搭建网络
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = nn.Linear(47*1,28)  #定义Linear的输入和输出的形状
        self.fc2 = nn.Linear(28,12)  #定义Linear的输入和输出的形状

    def forward(self,x):
        x = x.view(-1,47*1)  #对数据形状变形，-1表示该位置根据后面的形状自动调整
        x = self.fc1(x) #[batch_size,28]
        x = F.relu(x)  #[batch_size,28]
        x = self.fc2(x) #[batch_size,10]
        return F.log_softmax(x, dim=-1)

In [326]:
#准备迭代器
train_dataloader = torch.utils.data.DataLoader(data,batch_size=64,shuffle=True)

In [327]:
train_batch_size = 64
test_batch_size = 128
img_size = 47

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = lungdataset if train else testdataset
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

In [328]:
mnist_net = MnistNet().double()
optimizer = optim.Adam(mnist_net.parameters(),lr= 0.001)
train_loss_list = []
train_count_list = []
def train(epoch):
    mnist_net.train(True)
    train_dataloader = get_dataloader(True)
    print("开始训练：")
    for idx,(data,target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = mnist_net(data)
        loss = F.nll_loss(output,target) #对数似然损失
        loss.backward()
        optimizer.step()
        if idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,  idx * len(data), len(train_dataloader.dataset),100. * idx / len(train_dataloader), loss.item()))
            train_loss_list.append(loss.item())
            train_count_list.append(idx*train_batch_size+(epoch-1)*len(train_dataloader))
    print("结束训练。")

In [329]:
epoch = 200
for i in range(epoch):
    train(i)

开始训练：
Train Epoch: 0 [0/352 (0%)]	Loss: 2.523196
结束训练。
开始训练：
Train Epoch: 1 [0/352 (0%)]	Loss: 2.458304
结束训练。
开始训练：
Train Epoch: 2 [0/352 (0%)]	Loss: 2.382757
结束训练。
开始训练：
Train Epoch: 3 [0/352 (0%)]	Loss: 2.289013
结束训练。
开始训练：
Train Epoch: 4 [0/352 (0%)]	Loss: 2.280735
结束训练。
开始训练：
Train Epoch: 5 [0/352 (0%)]	Loss: 2.188329
结束训练。
开始训练：
Train Epoch: 6 [0/352 (0%)]	Loss: 2.159799
结束训练。
开始训练：
Train Epoch: 7 [0/352 (0%)]	Loss: 2.150167
结束训练。
开始训练：
Train Epoch: 8 [0/352 (0%)]	Loss: 2.011738
结束训练。
开始训练：
Train Epoch: 9 [0/352 (0%)]	Loss: 2.007132
结束训练。
开始训练：
Train Epoch: 10 [0/352 (0%)]	Loss: 1.851756
结束训练。
开始训练：
Train Epoch: 11 [0/352 (0%)]	Loss: 1.938504
结束训练。
开始训练：
Train Epoch: 12 [0/352 (0%)]	Loss: 1.958335
结束训练。
开始训练：
Train Epoch: 13 [0/352 (0%)]	Loss: 1.918250
结束训练。
开始训练：
Train Epoch: 14 [0/352 (0%)]	Loss: 1.845027
结束训练。
开始训练：
Train Epoch: 15 [0/352 (0%)]	Loss: 1.791679
结束训练。
开始训练：
Train Epoch: 16 [0/352 (0%)]	Loss: 1.768000
结束训练。
开始训练：
Train Epoch: 17 [0/352 (0%)]	Loss: 1.674881
结束训练。
开始

In [330]:
def test():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=False)
    with torch.no_grad():
        for data, target in test_dataloader:
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [331]:
test()


Test set: Avg. loss: 2.1703, Accuracy: 21/88 (23.86%)



In [332]:
torch.save(mnist_net.state_dict(),"model/mnist_net.pt") #保存模型参数
torch.save(optimizer.state_dict(), 'results/mnist_optimizer.pt') #保存优化器参数

FileNotFoundError: [Errno 2] No such file or directory: 'model/mnist_net.pt'

In [ ]:
mnist_net.load_state_dict(torch.load("model/mnist_net.pt"))
optimizer.load_state_dict(torch.load("results/mnist_optimizer.pt"))